# Import dependencies

In [ ]:
!pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:00:00


In [ ]:
!apt-get update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,118 kB]
Fetched 2,351 kB in 2s (963 kB/s)
Reading package lists... Done


In [ ]:
!apt-get install -y chromium-browser

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  apparmor libfuse3-3 liblzo2-2 libudev1 snapd squashfs-tools systemd-hwe-hwdb udev
Suggested packages:
  apparmor-profiles-extra apparmor-utils fuse3 zenity | kdialog
The following NEW packages will be installed:
  apparmor chromium-browser libfuse3-3 liblzo2-2 snapd squashfs-tools systemd-hwe-hwdb udev
The following packages will be upgraded:
  libudev1
1 upgraded, 8 newly installed, 0 to remove and 44 not upgraded.
Need to get 28.3 MB of archives.
After this operation, 117 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 apparmor amd64 3.0.4-2ubuntu2.3 [595 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 liblzo2-2 amd64 2.10-2build3 [53.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 squashfs-tools amd64 1:4.5-3build1 [159 kB]
Get:4 http://archive.ubuntu

In [ ]:
!apt install chromium-chromedriver

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  chromium-chromedriver
0 upgraded, 1 newly installed, 0 to remove and 44 not upgraded.
Need to get 2,308 B of archives.
After this operation, 77.8 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 chromium-chromedriver amd64 1:85.0.4183.83-0ubuntu2.22.04.1 [2,308 B]
Fetched 2,308 B in 0s (12.3 kB/s)
Selecting previously unselected package chromium-chromedriver.
(Reading database ... 122377 files and directories currently installed.)
Preparing to unpack .../chromium-chromedriver_1%3a85.0.4183.83-0ubuntu2.22.04.1_amd64.deb ...
Unpacking chromium-chromedriver (1:85.0.4183.83-0ubuntu2.22.04.1) ...
Setting up chromium-chromedriver (1:85.0.4183.83-0ubuntu2.22.04.1) ...


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from tqdm import tqdm

# Parsing Air Alarms (Tseva Adom) data

In [ ]:
!ls /usr/lib/chromium-browser/chromedriver

/usr/lib/chromium-browser/chromedriver


In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--diable-dve-shm-uage')


driver = webdriver.Chrome(options=options)

In [ ]:
def flatten_list(nested_list):
    flat_list = []
    for item in nested_list:
        if isinstance(item, list):
            flat_list.extend(flatten_list(item))
        else:
            flat_list.append(item)
    return flat_list


In [ ]:
# 2383
for i in tqdm(range(101, 103)):
  driver.get(f"https://www.tzevaadom.co.il/en/alerts/{i}")
  time.sleep(1)
  with open(f"page/page_{i}.html", "w", encoding='utf-8') as f:
      f.write(driver.page_source)

100%|██████████| 2/2 [00:02<00:00,  1.12s/it]


In [ ]:
with open('page_1.html', 'r', encoding='utf-8') as file:
    html_content = file.read()

# Parse the HTML content
soup = BeautifulSoup(html_content, 'html.parser')

In [ ]:
title_tag = soup.find("div", class_="header").text.replace('Alert - ', '')
title_tag

'20/05/2021'

In [ ]:
column_names = ['date', 'bomb_time', 'region', 'place', 'attack_number']
dataset_bombs = pd.DataFrame(columns=column_names)
dataset_bombs

,date,bomb_time,region,place,attack_number


In [ ]:
for k in tqdm(range(1, 2383)):
  with open(f'page/page_{k}.html', 'r', encoding='utf-8') as file:
      html_content = file.read()
  # Parse the HTML content
  soup = BeautifulSoup(html_content, 'html.parser')

  title_tag = soup.find("div", class_="header").text.replace('Alert - ', '')
  for i in soup.find_all('div', class_='alert-info-card'):
    # Date
    date = title_tag
    # Time of bombardment
    times = i.find('h4', class_='time').find('div').text.replace(' ', '')
    # Region
    region = i.find('div', class_='polygon').find('h3').text
    # Cities
    cities = list(map(lambda x: x.text, i.find_all('div', class_='polygon-number')))
    final_cities = flatten_list(list(map(lambda x: x.split(', '), cities)))
    for city in final_cities:
      new_pd = pd.DataFrame([[date, times, region, city, k]], columns=column_names)
      dataset_bombs= pd.concat([dataset_bombs, new_pd], axis=0, ignore_index=True)

100%|██████████| 2382/2382 [01:23<00:00, 28.51it/s]


In [ ]:
cities = list(map(lambda x: x.text, soup.find_all('div', class_='alert-info-card')[0].find_all('div', class_='polygon-number')))
final_cities = flatten_list(list(map(lambda x: x.split(', '), cities)))

In [ ]:
dataset_bombs

,date,bomb_time,region,place,attack_number
0,20/05/2021,18:00:37,Gaza Envelope,Mefalsim,1
1,20/05/2021,18:00:37,Gaza Envelope,Nir Am Shooting Range,1
2,20/05/2021,18:00:37,Gaza Envelope,Sderot,1
3,20/05/2021,18:00:37,Gaza Envelope,Ibim,1
4,20/05/2021,18:00:37,Gaza Envelope,Nir Am,1
...,...,...,...,...,...
16266,04/04/2024,17:43:25,Western Negev,Sharsheret,2382
16267,04/04/2024,17:43:34,Gaza Envelope,Zimrat,2382
16268,04/04/2024,17:43:34,Gaza Envelope,Shuva,2382
16269,04/04/2024,17:43:34,Gaza Envelope,Kfar Maimon and Tushia,2382


In [ ]:
dataset_bombs['date'].unique()

array(['20/05/2021', '17/06/2021', '20/07/2021', '04/08/2021',
       '06/08/2021', '16/08/2021', '10/09/2021', '11/09/2021',
       '12/09/2021', '18/10/2021', '03/11/2021', '07/02/2022',
       '08/02/2022', '18/02/2022', '16/03/2022', '17/04/2022',
       '18/04/2022', '20/04/2022', '19/05/2022', '18/06/2022',
       '14/07/2022', '15/07/2022', '16/07/2022', '27/07/2022',
       '05/08/2022', '06/08/2022', '07/08/2022', '03/11/2022',
       '26/01/2023', '27/01/2023', '01/02/2023', '02/02/2023',
       '05/02/2023', '11/02/2023', '13/02/2023', '23/02/2023',
       '07/03/2023', '08/03/2023', '09/03/2023', '18/03/2023',
       '05/04/2023', '06/04/2023', '07/04/2023', '08/04/2023',
       '02/05/2023', '03/05/2023', '10/05/2023', '11/05/2023',
       '12/05/2023', '13/05/2023', '14/05/2023', '21/05/2023',
       '26/05/2023', '18/06/2023', '20/06/2023', '04/07/2023',
       '06/07/2023', '08/07/2023', '15/07/2023', '22/07/2023',
       '29/07/2023', '19/09/2023', '07/10/2023', '08/10

In [ ]:
dataset_bombs['date'] = pd.to_datetime(bombs['date'], format='%d/%m/%Y')

dataset_bombs.to_csv('air_alarms_bombardments.csv', index_col=0)

# Casualties since October 7, 2023

## Civilian Casualties dataset creation

In [ ]:
import codecs
from bs4 import BeautifulSoup

# Open the HTML file with a specified encoding
with codecs.open("swords_of_iron.html", 'r', 'utf-8') as file:
    html_content = file.read()

# Parse the HTML content
soup = BeautifulSoup(html_content, 'lxml')

# Example: find an element by tag or id
element = soup.find('div', id='htmlContent_0')

# Print the found element or a message if not found
print(element)

<div class="mt-2 html-section" id="htmlContent_0"><p>The names of over 800 civilians who were murdered in their homes, towns and communities confronting Hamas terrorists during the brutal attack on October 7, 2023, as well as subsequent attacks after October 7 have been released by Israeli authorities. This list will be updated with additional information as it is released.</p><p><em>(This list does not include IDF personnel, Israel Police officers, Israel Security Agency officers and first responders who were killed in the attack).</em></p><p><em><strong>Click on a link below for victims' names:</strong></em></p><ul><li><a href="https://www.gov.il/en/pages/swords-of-iron-civilian-casualties#Civilian%20casualties"><strong>Civilian casualties </strong></a></li><li><strong><a href="https://www.gov.il/en/pages/swords-of-iron-civilian-casualties#NOVA">Nova Music Festival victims</a></strong></li><li><a href="https://www.gov.il/en/pages/swords-of-iron-civilian-casualties#ZIKIM"><strong>Ziki

In [ ]:
names = []
for i in range(3, 830):
  names.append(element.find('table').find_all('li')[i].text.split(',')[0].split(' of ')[0])

In [ ]:
ages = []
for i in range(3, 830):
  if (len(element.find('table').find_all('li')[i].text.split(','))) > 1:
    ages.append(element.find('table').find_all('li')[i].text.split(',')[1].split(' of ')[0])
  else:
    ages.append(np.nan)

In [ ]:
place = []
for i in range(3, 830):
  if len(element.find('table').find_all('li')[i].text.split(', of ')) > 1:
    place.append(element.find('table').find_all('li')[i].text.split(', of ')[1].split(",")[0].split(".")[0].split(' - ')[0].split(' – ')[0].split('– ')[0])
  else:
    place.append('Unknown')

In [ ]:
nationality = []
for i in range(3, 830):
  if (len(element.find('table').find_all('li')[i].text.split('national'))) > 1:
    nationality.append(element.find('table').find_all('li')[i].text.split('national')[0].split(' ')[-2])
  else:
    nationality.append('Israeli')

In [ ]:
casualties_swords = pd.DataFrame({'name': names, 'age': ages, 'place': place, 'nationality': nationality})
casualties_swords.head(10)

,name,age,place,nationality
0,Sharif Suad,35,the Bedouin town of Sallama near Karmiel,Israeli
1,Benjamin Achimeir,14,the West Bank outpost Malachei HaShalom,Israeli
2,Lidor Levy,34,Gan Yavne,Israeli
3,Elad Katzir,47,Kibbutz Nir Oz,Israeli
4,Uriel Baruch,35,Unknown,Israeli
5,Zaher Bashara,38,the Druze village Ein Qiniyye,Israeli
6,Uri Moyal,51,Dimona,Israeli
7,Patnibin Maxwell,31,Unknown,Indian
8,Yitzhak Zeiger,57,Unknown,Israeli
9,Oriya Hartom,16,Dolev,Israeli


In [ ]:
casualties_swords.to_csv("civilian_casualties.csv")

In [ ]:
# Open the HTML file with a specified encoding
with codecs.open("swords_of_iron.html", 'r', 'utf-8') as file:
    html_content = file.read()

# Parse the HTML content
soup = BeautifulSoup(html_content, 'lxml')

# Example: find an element by tag or id
element = soup.find('div', id='htmlContent_1')

# Print the found element or a message if not found
print(element)

<div class="mt-2 html-section" id="htmlContent_1"><table border="0" cellpadding="1" cellspacing="1"><tbody><tr><td><a id="NOVA" name="NOVA"><strong>NOVA MUSIC FESTIVAL VICTIMS</strong></a></td></tr><tr><td><ul><li>The body of Elyakim Libman, 24, previously thought to be one of the Israeli hostages held in Gaza, was found recently in the grave of another victim of the October 7 attacks, in Israeli territory. Elyakim was working as a security guard at the Nova Musical Festival, where he managed to assist in the evacuation of many of the partygoers. On Friday, 3 May, the authorities informed the family that Elyakim was murdered during the Hamas' attack on October 7 and his body had been identified.</li><li>Gal Danguri, 23, of Beit Arye - Murdered by Hamas terrorists at the Nova Music Festival at Kibbutz Re'im.</li></ul></td></tr><tr><td><ul><li>Shmil Abasov, 33, of Beer Sheva - Murdered by Hamas terrorists while attempting to rescue partygoers at the Nova Music Festival at Kibbutz Re'im.<

In [ ]:
# Open the HTML file with a specified encoding
with codecs.open("swords_of_iron.html", 'r', 'utf-8') as file:
    html_content = file.read()

# Parse the HTML content
soup = BeautifulSoup(html_content, 'lxml')

# Example: find an element by tag or id
element = soup.find('div', id='htmlContent_1')

# Print the found element or a message if not found
print(element)


names = []
for i in range(2, 371):
  names.append(element.find('table').find_all('li')[i].text.split(',')[0].split(' of ')[0])

ages = []
for i in range(2, 371):
  if (len(element.find('table').find_all('li')[i].text.split(','))) > 1:
    ages.append(element.find('table').find_all('li')[i].text.split(',')[1].split(' of ')[0])
  else:
    ages.append(np.nan)

place = []
for i in range(2, 371):
  if len(element.find('table').find_all('li')[i].text.split(', of ')) > 1:
    place.append(element.find('table').find_all('li')[i].text.split(', of ')[1].split(",")[0].split(".")[0].split(' - ')[0].split(' – ')[0].split('– ')[0])
  else:
    place.append('Unknown')

nationality = []
for i in range(2, 371):
  if (len(element.find('table').find_all('li')[i].text.split('national'))) > 1:
    nationality.append(element.find('table').find_all('li')[i].text.split('national')[0].split(' ')[-2])
  else:
    nationality.append('Israeli')

casualties_music_fest = pd.DataFrame({'name': names, 'age': ages, 'place': place, 'nationality': nationality})
casualties_music_fest.head(10)


<div class="mt-2 html-section" id="htmlContent_1"><table border="0" cellpadding="1" cellspacing="1"><tbody><tr><td><a id="NOVA" name="NOVA"><strong>NOVA MUSIC FESTIVAL VICTIMS</strong></a></td></tr><tr><td><ul><li>The body of Elyakim Libman, 24, previously thought to be one of the Israeli hostages held in Gaza, was found recently in the grave of another victim of the October 7 attacks, in Israeli territory. Elyakim was working as a security guard at the Nova Musical Festival, where he managed to assist in the evacuation of many of the partygoers. On Friday, 3 May, the authorities informed the family that Elyakim was murdered during the Hamas' attack on October 7 and his body had been identified.</li><li>Gal Danguri, 23, of Beit Arye - Murdered by Hamas terrorists at the Nova Music Festival at Kibbutz Re'im.</li></ul></td></tr><tr><td><ul><li>Shmil Abasov, 33, of Beer Sheva - Murdered by Hamas terrorists while attempting to rescue partygoers at the Nova Music Festival at Kibbutz Re'im.<

,name,age,place,nationality
0,Shmil Abasov,33,Beer Sheva,Israeli
1,Eden Abdulayev,23,Ramat Gan,Israeli
2,Alexandre Look,35,Montreal,Canadian
3,Jake Marlo,26,Unknown,British
4,Orel Abuhazera,25,Unknown,Israeli
5,Amar Abu Sebila,25,Abu Talul,Israeli
6,Eitan Levy,53,Bat Yam,Israeli
7,Shani Nicol Louk,23,Srigim,Israeli
8,Noy Aviv,29,Eilat,Israeli
9,Dor Avitan,26,Eilat,Israeli


In [ ]:
casualties_music_fest.to_csv('casualties_music_fest.csv')

## IDF Casualties

In [ ]:
# dates
dates = []

for i in range(123):
  for j in range(len(element.find('table').find_all('tr')[i].find_all('li'))):
    if (j != 1) and (i != 103):
      dates.append(element.find('table').find_all('tr')[i].find('strong').text)

In [ ]:
# ages
ages = []
for i in range(123):
  for j in range(len(element.find('table').find_all('tr')[i].find_all('li'))):
    if (j != 1) and (i != 103):
      ages.append(element.find('table').find_all('tr')[i].find_all('li')[j].text.split(',')[1].split(' ')[-1])

In [ ]:
# rank and name
data = []
for i in range(123):
  for j in range(len(element.find('table').find_all('tr')[i].find_all('li'))):
    if (j != 1) and (i != 103):
      data.append(element.find('table').find_all('tr')[i].find_all('li')[j].text.split(',')[0].split(' '))

In [ ]:
hometown = []
for i in range(123):
  for j in range(len(element.find('table').find_all('tr')[i].find_all('li'))):
    if (j != 1) and (i != 103):
      if len(element.find('table').find_all('tr')[i].find_all('li')[j].text.split('of ')) > 1:
        hometown.append(element.find('table').find_all('tr')[i].find_all('li')[j].text.split('of ')[1].split("- ")[0].split(' – ')[0].split('–')[0].split(' –')[0])
      elif len(element.find('table').find_all('tr')[i].find_all('li')[j].text.split('Of ')) > 1:
        hometown.append(element.find('table').find_all('tr')[i].find_all('li')[j].text.split('Of ')[1].split("- ")[0].split(' – ')[0].split('–')[0].split(' –')[0])
      else:
        hometown.append('Unknown')

In [ ]:
rank_terms = {
    'Staff', 'Sgt.', 'Master', 'Sgt.', 'Maj.', 'Captain', 'Warrant', 'Officer', 'Command',
    'Sergeant', 'Capt.', 'Maj.', '1st', 'Class', 'res.', 'Command', 'Sergeant', 'Lt.',
    'Col.', 'Cpl.', 'Corporal', 'Lieutenant', 'Lt.-Col.', 'Maj.-Gen.', 'Brig.', 'Gen.',
    'Admiral', 'Commodore', 'Rear', 'Adm.', 'Vice', 'Sgt', 'Major', 'Maj', 'General',
    'Lieutenant', 'Colonel', 'Second', 'Lieutenant', 'First', 'Lieutenant', 'Sergeant',
    'Major', 'Chief', 'Warrant', 'Officer', 'Private', 'Specialist', 'Corporal', 'Master',
    'Sgt.', 'Sgt', '1st', 'Class', '(res.)', 'Lt', 'Col', '\xa0Maj.', 'Master-Sgt.',
    'Staff-Sgt.', 'Master', 'Sergeant', 'Master-Sgt.', 'Capt.', '(res.)', 'Lt.-Col.',
    'Maj.-Gen.', 'Chief', 'Warrant', 'Officer', 'Sergeant', 'first', 'class', 'Master',
    'sergeant', 'Sgt.', 'Major', 'Col.', 'Staff', 'sergeant'
}

# Initialize lists to hold names and ranks
names = []
ranks = []

# Function to extract names and ranks
def extract_names_ranks(data):
    for entry in data:
        current_rank = []
        current_name = []
        is_rank = True  # Assume the first element(s) are part of the rank

        for element in entry:
            if element in rank_terms or any(sub in element for sub in ['(', 'res.', 'Class', '1st', 'Col', 'Maj', 'Gen', 'Adm', 'Sgt', 'Capt', 'Lt', 'Private', 'Specialist', 'Corporal']):
                current_rank.append(element)
            else:
                current_name.append(element)

        # Join elements to form the full rank and name
        full_rank = ' '.join(current_rank)
        full_name = ' '.join(current_name)

        # Append to lists
        ranks.append(full_rank)
        names.append(full_name)

# Call the function with the dataset
extract_names_ranks(data)

# Output the lists
print("Names:", names)
print("Ranks:", ranks)

Names: ['Haim Sabach', 'Dan Kamkagi', 'Michael Ruzal', 'Ido Testa', 'Ruben Marc Mordechai Assouline', 'Ido Aviv', 'Salm Alkreshat', 'Dor Zimel', 'Reef Harush', 'Ido Baruch', 'Amitai Even Shoshan', 'Nadav Cohen', 'Alon Kudriashov', 'Nissim Kachlon', 'Lior Raviv', 'Ilay David Garfinkel', 'Sebastian Haion', 'Matan Vinogradov', 'Daniel Perez', 'Uri Moyal', 'Itay Hen', 'Michael Gal', 'Amishar Ben David', 'David Sasson', 'Dennis Yekimov', 'Inon Itzhak', 'Dolev Malca', 'Itai Seif', 'Oz Daniel', 'Ido Eli Zrihen', 'Eyal Shuminov', 'Avraham Wovagen', 'Maoz Morell', 'Simon Shlomov', 'Ori Yaish', 'Noam Haba', 'Omer Sarah Benjo', 'Yair Cohen', 'Ziv Chen', 'Adi Eldor', 'Hanan Drori', 'David Shakuri', 'Shimon Yehoshua Asulin', 'Yitzhar Hofman', 'Yuval Nir', 'Netzer Simchi', 'Eliran Yeger', 'Zegeye Daniel Kasau', 'Itamar Tal', 'Adam Bismut', 'Ahmad Abu Latif', 'Ariel Mordechay Wollfstal', 'Elkana Vizel', 'Nir Binyamin', 'Sagi Idan', 'Mark Kononovich', 'Israel Socol', 'Rafael Elias Mosheyoff', 'Barak H

In [ ]:
idf_casualties = pd.DataFrame({'name': names, 'age': ages, 'hometown': hometown, 'rank': ranks, 'date': dates})
idf_casualties.head(5)

,name,age,hometown,rank,date
0,Haim Sabach,20,Holon,Staff Sgt.,8 May 2024
1,Dan Kamkagi,31,Kfar HaOranim,Master Sgt. (res.),6 May 2024
2,Michael Ruzal,18,Rishon LeZion,Sgt.,5 May 2024
3,Ido Testa,19,Jerusalem,Staff Sgt.,5 May 2024
4,Ruben Marc Mordechai Assouline,18,Ranana,Staff Sgt.,5 May 2024


In [ ]:
# standardize ranks
def standardize_ranks(rank):
    rank = rank.replace('\xa0', '').strip()
    rank = rank.replace('Capt.', 'Captain')
    rank = rank.replace('Sgt.', 'Sergeant')
    rank = rank.replace('Maj.', 'Major')
    rank = rank.replace('Lt.', 'Lieutenant')
    rank = rank.replace('Col.', 'Colonel')
    rank = rank.replace('Cpl.', 'Corporal')
    rank = rank.replace('Staff-Sgt.', 'Staff Sergeant')
    rank = rank.replace('Staff Sgt.', 'Staff Sergeant')
    rank = rank.replace('1st', 'First')
    rank = rank.replace('Sgt. Major', 'Sergeant Major')
    rank = rank.replace('Sgt. Maj.', 'Sergeant Major')
    rank = rank.replace('Master-Sgt.', 'Master Sergeant')
    rank = rank.replace('Master Sgt.', 'Master Sergeant')
    rank = rank.replace('res.', 'Reserve')
    rank = rank.replace('Sergeant first class', 'Sergeant First Class')
    rank = rank.replace('Command Sgt. Maj.', 'Command Sergeant Major')
    rank = rank.replace('Lt.-Col.', 'Lieutenant Colonel')
    rank = rank.replace('Staff-Sergeant', 'Staff Sergeant')
    rank = rank.replace('.', '').strip()  # Remove periods and strip whitespace
    rank = rank.replace('sergeant', 'Sergeant').replace('Sergeant First class', 'Sergeant First Class')
    rank = rank.replace('Staff sergeant', 'Staff Sergeant')
    rank = rank.replace('Lieutenant-Colonel', 'Lieutenant Colonel')
    rank = rank.replace('Master-Sergeant', 'Master Sergeant')
    rank = rank.replace('Staff-Sergeant', 'Staff Sergeant')
    rank = rank.replace('  ', ' ')  # Replace double spaces with single if any
    # Handle specific cases missed by general rules
    if 'Master SergeantReserve' in rank:
        rank = 'Master Sergeant Reserve'
    if 'Sergeant First ClassReserve' in rank:
        rank = 'Sergeant First Class Reserve'
    if 'Sergeant First. Class' in rank:
        rank = 'Sergeant First Class Reserve'
    return rank
    return rank

# Apply the cleaning function
idf_casualties['rank'] = idf_casualties['rank'].apply(standardize_ranks)

# Print unique ranks to verify standardization
print(idf_casualties['rank'].unique())

['Staff Sergeant' 'Master Sergeant (Reserve)' 'Sergeant'
 'Sergeant First Class' 'Major (Reserve)' 'Captain' 'Sergeant Major'
 'Warrant Officer (Reserve)' 'Command Sergeant Major'
 'Sergeant First Class (Reserve)' 'Sergeant Major (Reserve)' 'Major'
 'Captain (Reserve)' 'Master Sergeant' 'Lieutenant Colonel' 'Lieutenant'
 'Master Sergeant(Reserve)' 'Master Sergeant First Class (Reserve)'
 'Corporal' 'Colonel' 'Lieutenant (Reserve)' 'Captain Master'
 'Sergeant First Class(Reserve)' 'Staff Sergeant (Reserve)'
 'Sergeant (Reserve)' 'Colonel (Reserve)' 'Warrant Officer'
 'Chief Warrant Officer (Reserve)' 'Chief Warrant Officer']


In [ ]:
idf_casualties['is_reserve'] = idf_casualties['rank'].str.contains(r'\(Reserve\)').astype(int)

idf_casualties

,name,age,hometown,rank,date,is_reserve
0,Haim Sabach,20.0,Holon,Staff Sergeant,8 May 2024,0
1,Dan Kamkagi,31.0,Kfar HaOranim,Master Sergeant (Reserve),6 May 2024,1
2,Michael Ruzal,18.0,Rishon LeZion,Sergeant,5 May 2024,0
3,Ido Testa,19.0,Jerusalem,Staff Sergeant,5 May 2024,0
4,Ruben Marc Mordechai Assouline,18.0,Ranana,Staff Sergeant,5 May 2024,0
...,...,...,...,...,...,...
541,Chen Bouchris,26.0,Ashdod,Major,7 October 2023,0
542,Or Yossef Ran,29.0,Itamar,Major,7 October 2023,0
543,Adi Groman,19.0,Moshav Hogla,Sergeant,7 October 2023,0
544,Ido Edri,24.0,Moshav Gibton,Lieutenant (Reserve),7 October 2023,1


In [ ]:
# delete (Reserve)
idf_casualties['rank'] = idf_casualties['rank'].apply(lambda x: x.replace(' (Reserve)', '')).apply(lambda x: x.replace('(Reserve)', ''))
idf_casualties['rank'].unique()

array(['Staff Sergeant', 'Master Sergeant', 'Sergeant',
       'Sergeant First Class', 'Major', 'Captain', 'Sergeant Major',
       'Warrant Officer', 'Command Sergeant Major', 'Lieutenant Colonel',
       'Lieutenant', 'Master Sergeant First Class', 'Corporal', 'Colonel',
       'Captain Master', 'Chief Warrant Officer'], dtype=object)

In [ ]:
idf_casualties['rank'].value_counts().sort_values(ascending=False)

rank
Staff Sergeant                 147
Sergeant                        85
Captain                         58
Sergeant First Class            54
Major                           51
Sergeant Major                  48
Master Sergeant                 45
Lieutenant                      16
Corporal                        15
Lieutenant Colonel               9
Warrant Officer                  8
Colonel                          4
Master Sergeant First Class      2
Chief Warrant Officer            2
Command Sergeant Major           1
Captain Master                   1
Name: count, dtype: int64

In [ ]:
def standardize_hometowns(hometown):
    # Strip trailing spaces
    if type(hometown) == type(idf_casualties_final['hometown'][0]):
      clean_hometown = hometown.strip()
    else:
      return hometown

    # Correct common spelling mistakes and inconsistencies
    corrections = {
        'Rishon LeZion ': 'Rishon LeZion',
        'Modi’in': 'Modiin',
        'Rosh Ha’ayin': 'Rosh HaAyin',
        'Haifa ': 'Haifa',
        'Jerusalem ': 'Jerusalem',
        'Tel Aviv ': 'Tel Aviv',
        'Tel Aviv-Yaffo ': 'Tel Aviv-Yafo',
        'beer Sheva ': 'Be’er Sheva',
        'Be’er Sheva': 'Beer Sheva',
        'Kfar Saba ': 'Kfar Saba',
        'Ranana ': 'Ra’anana',
        'Eilat ': 'Eilat',
        'Afula ': 'Afula',
        'Hadera ': 'Hadera',
        'Netanya ': 'Netanya',
        'Rehovot ': 'Rehovot',
        'Kfar Yona ': 'Kfar Yona',
        'Herzliya ': 'Herzliya',
        'Modi’in ': 'Modiin',
        'Ramat Hasharon ': 'Ramat Hasharon',
        'Ashkelon ': 'Ashkelon',
        'Beit Shean ': 'Beit Shean',
        'Sderot ': 'Sderot',
        'Modi’in-Maccabim-Reut ': 'Modiin-Maccabim-Reut',
        'Kibbutz Kfar Aza ': 'Kibbutz Kfar Aza',
        'Tel Aviv-Yaffo': 'Tel Aviv',
    }

    # Apply corrections
    if clean_hometown in corrections:
        return corrections[clean_hometown]
    return clean_hometown

# Apply the standardization function to the 'hometown' column
idf_casualties['hometown'] = idf_casualties['hometown'].apply(standardize_hometowns)

# Print the result to verify
print(idf_casualties['hometown'].unique())

['Holon' 'Kfar HaOranim' 'Rishon LeZion' 'Jerusalem' 'Ranana' 'Carmiel'
 'Abu-Rabia' 'Even Yehuda' 'Kibbutz Ramat David' 'Tel Mond'
 "Moshav Azri'el" 'Haifa' 'Modiin' 'Hadera' 'Moshav Sitria' 'Rosh HaAyin'
 'Yad Binyamin' 'Dimona' 'Netanya' 'Eli' 'Ganot Hadar' 'Beer Sheva'
 'Mitzpe Ramon' 'Shlomi' 'Yeruham' 'Kfar Saba' 'Karmiel' 'Talmon'
 'Kiriat Bialik' "Moshav Ge'a" 'Ramat Gan' 'Rehovot' 'Beit Shemesh'
 'Eshhar' 'Kfar Etzion' 'Masad' 'Tel Aviv' 'Yokneam Illit'
 'Kibbutz Mesilot' 'Karnei Shomron' 'Rahat' 'Elazar' 'Bnei Dekalim'
 'Givatayim' 'Herzliya' 'Pardes Hanna-Karkur' 'Yehud-Monosson'
 'Kiriat Arba' 'Moshav Mevo Beitar' 'Alon Shvut' 'Moshav Beit Gamliel'
 'Moshav Nehalim' 'Hurfeish' 'Moshav Kfar Yuval' 'Kiriat Gat' 'Raanana'
 'Efrat' 'Hod Hasharon' 'Kibbutz Ramat Yohanan' 'Kfar Yehoshua' 'Maghar'
 'Tiberias' 'Petah Tikva' 'Kokhav Ya’akov' 'Kfar Yona' 'Kiriat Motzkin'
 'Neve Daniel' 'Merkaz Shapira' 'Kibbutz Mishmarot' 'Moshav Tzofar'
 'Lapid' 'Kibbutz Beit Keshet' 'Ashkelon' 'Has

In [ ]:
idf_casualties.to_csv('idf_casualties.csv')

In [ ]:
towns = idf_casualties_final['hometown'].unique()

In [ ]:
!pip install geopy

In [ ]:
# Import the necessary modules for geocoding
from geopy.geocoders import Nominatim
import time

df1 = pd.DataFrame(towns, columns=['town'])

# Initialize the geolocator
geolocator = Nominatim(user_agent="geo-coordinates")

# Function to fetch coordinates for a city
def get_coordinates(city):
    try:
        location = geolocator.geocode(city)
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except Exception:
        return None, None

# Add two new columns for latitude and longitude
df1['latitude'], df1['longitude'] = zip(*df1['town'].apply(lambda x: get_coordinates(x)))

df1.to_csv('coordinates_towns.csv')

## Civilian Results

In [ ]:
casualties_swords.to_csv("casualties_civilians.csv")

In [ ]:
civilians = pd.read_csv('civilians.csv', names=['civ'], sep = ';')
civ = list(civilians['civ'])

In [ ]:
i = 1
killed_date = []
while i < len(civ):
  killed_date.append(civ[i].split('Killed: ')[-1])
  i += 3

In [ ]:
i = 0
name = []
while i < len(civ):
  name.append(civ[i].split(', ')[0])
  i+=3

In [ ]:
i = 2
killed_place = []
while i < len(civ):
  killed_place.append(civ[i].split('Location: ')[-1])
  i += 3

In [ ]:
civilians_data = pd.DataFrame({"name": name, "killed_date": killed_date, "killed_place": killed_place})
civilians_data

,name,killed_date,killed_place
0,Yossi Sharabi,"After October 7, 2023",Murdered in captivity by Hamas
1,Edna Bluestein,"January 15, 2024",Raanana (Ramming and stabbing attack)
2,Itay Svirsky,"After October 7, 2023",Murdered in captivity by Hamas
3,Mira Ayalon,"January 4, 2024",Moshav Kfar Yuval (Anti-tank missile attack)
4,Tamir Adar,"October 7, 2023",Abducted and murdered on the day of the attack
...,...,...,...
653,Avi Mor,"October 7, 2023",In his home
654,Maayan Mor,"October 7, 2023","Nova Music Festival, Kibbutz Re'im"
655,Raffi Mordo,"October 7, 2023",In his home
656,Ya’akov Mortov,Date not specified,While on a tour bus that stopped in Sderot
